In [1]:
import sys

sys.path.insert(0, "..")

### Setting here


In [2]:
# Paths
OFFLINE_DATASET_DIR = "D:/storage/odl/cache/offline/reintel2020/dataset.pt"
CHECKPOINT_DIR = "D:/storage/odl/checkpoints/offline/reintel2020/TextCNN_WE/"

# Hyperparameters for TextCNN with trained-WE
MAX_LENGTH = 65
EMBEDDING_DIM = 300
NUM_FILTERS = 100
FILTER_SIZES = [3, 4, 5]
MAX_FEATURES = 100000
LR = 2e-4

### Load dataset from cache


In [3]:
import torch

dataset = torch.load(OFFLINE_DATASET_DIR)
train_dataset = [
    {k: v for k, v in d.items() if k in ["post_message", "label"]}
    for d in dataset["train"].data
]
test_dataset = [
    {k: v for k, v in d.items() if k in ["post_message", "label"]}
    for d in dataset["test"].data
]

del dataset

len(train_dataset), len(test_dataset)

(3497, 875)

### Define model


In [4]:
from module import TextCNN, TokenEmbedding
import torch.nn as nn


def create_model(device):
    G = TokenEmbedding(MAX_LENGTH, EMBEDDING_DIM, MAX_FEATURES, device=device).to(
        device
    )
    F = TextCNN(EMBEDDING_DIM, NUM_FILTERS, FILTER_SIZES).to(device)

    optimizer = torch.optim.Adam(list(G.parameters()) + list(F.parameters()), lr=LR)

    return G, F, optimizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


criterion = nn.CrossEntropyLoss()

G, F, optimizer = create_model(device)

### Train


In [ ]:
# from torch.utils.data import DataLoader
# import torch.nn as nn
# import torch
# from train_tools import *
# from sklearn.metrics import accuracy_score, f1_score
# from metrics import MeanMetric


# def train_batch(inputs, targets, G, F, criterion, optimizer, device, freeze_G=False):
#     if freeze_G:
#         G.eval()
#     else:
#         G.train()
#     F.train()

#     targets = targets.to(device)
#     vectors = G(inputs)
#     outputs = F(vectors)

#     loss = criterion(outputs, targets)

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     return loss.item(), targets.tolist(), outputs.argmax(dim=1).tolist()


# def eval_batch(inputs, targets, G, F, criterion, device):
#     G.eval()
#     F.eval()

#     targets = targets.to(device)
#     vectors = G(inputs)
#     outputs = F(vectors)

#     loss = criterion(outputs, targets)

#     return loss.item(), targets.tolist(), outputs.argmax(dim=1).tolist()


# def train_online(
#     train_dataset,
#     test_dataset,
#     batch_size,  # for test loader
#     G: nn.Module,
#     F: nn.Module,
#     criterion,
#     optimizer,
#     device,
#     verbose=True,
#     path="./",
# ):
#     train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
#     acc = MeanMetric()
#     for batch in tqdm(train_loader):
#         inputs = batch["post_message"]
#         targets = batch["label"]

#         loss, true, pred = eval_batch(inputs, targets, G, F, criterion, device)
#         acc.update(accuracy_score(true, pred))

#     print(acc.result())


# train_online(
#     train_dataset=train_dataset,
#     test_dataset=test_dataset,
#     batch_size=32,
#     G=G,
#     F=F,
#     criterion=criterion,
#     optimizer=optimizer,
#     device=device,
#     verbose=True,
#     path="./",
# )

In [5]:
from train_tools import train_offline

train_offline(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    batch_size=32,
    G=G,
    F=F,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=100,
    device=device,
    early_stopping=True,
    patience=5,
    verbose=True,
    delta=0.001,
    path=CHECKPOINT_DIR,
)

Epoch 1:   0%|          | 0/110 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 0 - train_loss: 0.4444 - train_acc: 0.8144 - train_f1: 0.5522 - test_loss: 0.4257 - test_acc: 0.8343 - test_f1: 0.4807
Validation loss decreased (inf --> 0.425727), f1 score increased (-inf --> 0.480746).  Saving model ...


Epoch 2:   0%|          | 0/110 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/28 [00:00<?, ?it/s]

...

Epoch 36:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 35 - train_loss: 0.0128 - train_acc: 0.9983 - train_f1: 0.9969 - test_loss: 0.5428 - test_acc: 0.8629 - test_f1: 0.6424
EarlyStopping counter: 4 out of 5 with f1 score 0.6424387718963686


Epoch 37:   0%|          | 0/110 [00:00<?, ?it/s]

Epoch 37:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 36 - train_loss: 0.0152 - train_acc: 0.9983 - train_f1: 0.9969 - test_loss: 0.5224 - test_acc: 0.8629 - test_f1: 0.6490
EarlyStopping counter: 5 out of 5 with f1 score 0.6489890885750963
Early stopping
